In [205]:
from ortools.sat.python import cp_model
import random
import re
import time
import plotly.graph_objects as go
import plotly.express as px
import itertools

**Conjuntos**

$$ \text{CLIENTES} = \{1, 2, \ldots, N\} $$
$$ \text{NODOS} = \{0\} \cup \text{CLIENTES} $$
$$ \text{CAMIONES} = \{1, 2, 3, 4, 5, 6, 7\} $$

In [206]:
N = 20
CLIENTES = list(range(1, N + 1))
NODOS = [0] + CLIENTES
CAMIONES = [1, 2, 3, 4, 5]

In [207]:
random.seed(142)

**Parámetros**

$$ \text{CAPACIDAD\_CAMION}_c = 70 \text{ para cada camión } c \in \text{CAMIONES} $$
$$ \text{DEMANDA}_{cl}: \text{Demanda del cliente } cl \in \text{CLIENTES} $$
$$ \text{DISTANCIAS}_{n1,n2}: \text{Distancia euclidiana entre } n1 \in \text{NODOS} \text{ y } n2 \in \text{NODOS} $$

In [208]:
CAPACIDAD_CAMION = {c: 70 for c in CAMIONES}
DEMANDAS = {c: random.randint(10, 20) for c in CLIENTES}
COORDENADAS = {n: {"x": random.randint(0, N * 10), "y": random.randint(0, N * 10)} for n in NODOS}
DISTANCIAS = {(n1, n2): ((COORDENADAS[n1]["x"] - COORDENADAS[n2]["x"]) ** 2 + (COORDENADAS[n1]["y"] - COORDENADAS[n2]["y"]) ** 2) ** 0.5 for n1 in NODOS for n2 in NODOS}

In [209]:
fig = go.Figure()

# Añadir punto de depósito en rojo
fig.add_trace(
    go.Scatter(
        x=[COORDENADAS[0]["x"]],
        y=[COORDENADAS[0]["y"]],
        mode="markers",
        name="Depósito",
        marker=dict(color="red", size=15),
    )
)

# Añadir puntos de clientes en negro
fig.add_trace(
    go.Scatter(
        x=[coord["x"] for coord in list(COORDENADAS.values())[1:]],
        y=[coord["y"] for coord in list(COORDENADAS.values())[1:]],
        mode="markers",
        name="Clientes",
        marker=dict(color="black", size=10),
    )
)

# Añadir anotaciones de clientes
for cliente in CLIENTES:
    coord_cliente_x = COORDENADAS[cliente]["x"]
    coord_cliente_y = COORDENADAS[cliente]["y"]
    fig.add_annotation(x=coord_cliente_x - 8, y=coord_cliente_y + 8, text=f"<b>C{cliente}: </b> {DEMANDAS[cliente]}", showarrow=False)

# Configurar el layout del gráfico
fig.update_layout(
    title=f"COORDENADAS NODOS",
    xaxis_title="X",
    yaxis_title="Y",
    xaxis=dict(range=[-10, N * 10 + 10]),
    yaxis=dict(range=[-10, N * 10 + 10]),
    width=800,
    height=600,
    template="ggplot2",
)

fig.show()

In [210]:
model = cp_model.CpModel()

**Variables**

$$ x_{n1, n2, c} = \begin{cases} 
1 & \text{Si el camión } c \in \text{CAMIONES} \text{ viaja de } n1 \in \text{NODOS} \text{ a } n2 \in \text{NODOS} \\ 
0 & \text{d.l.c.} 
\end{cases} $$
$$ u_{n, c} \in \mathbb{R}^{+} \text{: Carga acumulada del vehículo } c \text{ al llegar al nodo } n \in \text{NODOS} $$

In [211]:
# x = model.addVars(NODOS, NODOS, CAMIONES, vtype=GRB.BINARY, name="x")
x = {(nodo1, nodo2, camion): model.NewBoolVar(f"x_{nodo1}_{nodo2}_{camion}") for nodo1 in NODOS for nodo2 in NODOS for camion in CAMIONES if nodo1 != nodo2}
assign = {(nodo, camion): model.NewBoolVar(f"assign_{nodo}_{camion}") for nodo in NODOS for camion in CAMIONES}
used_car = {camion: model.NewBoolVar(f"used_{camion}") for camion in CAMIONES}

**Función Objetivo**
$$ \text{Minimizar } \sum_{c \in \text{CAMIONES}} \sum_{n1 \in \text{NODOS}} \sum_{n2 \in \text{NODOS}} \text{DISTANCIAS}_{n1, n2} \cdot x_{n1, n2, c} $$

In [212]:
# model.setObjective(gp.quicksum(DISTANCIAS[n1, n2] * x[n1, n2, c] for n1 in NODOS for n2 in NODOS for c in CAMIONES), GRB.MINIMIZE)
model.Minimize(sum(DISTANCIAS[n1, n2] * x[n1, n2, c] for n1 in NODOS for n2 in NODOS for c in CAMIONES if n1 != n2))

### Restricciones

In [213]:
for nodo in NODOS:
    expressions = [assign[nodo, camion] for camion in CAMIONES]
    model.AddAtLeastOne(expressions)

for nodo in NODOS:
    for camion in CAMIONES:
        model.Add(assign[nodo, camion] <= used_car[camion])
        
for camion in CAMIONES:
    model.Add(assign[0, camion] == used_car[camion])
    
    arcs = [(n1, n2, x[n1, n2, camion]) for n1 in NODOS for n2 in NODOS if n1 != n2] + [(nodo, nodo, assign[nodo, camion].Not()) for nodo in NODOS]
    model.AddCircuit(arcs)

    # model.Add(sum(x[nodo, cliente, camion] * DEMANDAS[cliente] for nodo in NODOS for cliente in CLIENTES if nodo != cliente) <= CAPACIDAD_CAMION[camion])
    # expressions = [v * data[i][2] for (i, c, c), v in assign.items() if c == cc]
    # expressions = [assign[nodo, camion] * DEMANDAS[nodo] for (nodo, camion_), var in assign.items() if camion == camion_ and nodo in CLIENTES]
    model.Add(sum(assign[cliente, camion] * DEMANDAS[cliente] for cliente in CLIENTES) <= CAPACIDAD_CAMION[camion])

In [214]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 20

In [215]:
status = solver.Solve(model)

In [216]:
status

2

In [217]:
status == cp_model.OPTIMAL

False

In [218]:
solver.ObjectiveValue()

1494.1623366508247

In [219]:
rutas = {}
for camion in CAMIONES:
    for cliente in CLIENTES:
        if solver.Value(x[0, cliente, camion]) > 0.9:
            print(f"==== Ruta camión {camion} ====")
            ruta = [0, cliente]
            r = "0 -> " + str(cliente) + " -> "

            cliente_actual = cliente
            while solver.Value(x[cliente_actual, 0, camion]) < 0.9:
                for siguiente_cliente in CLIENTES:
                    if cliente_actual != siguiente_cliente:
                        if solver.Value(x[cliente_actual, siguiente_cliente, camion]) > 0.9:
                            break
                ruta.append(siguiente_cliente)
                r += str(siguiente_cliente) + " -> "
                cliente_actual = siguiente_cliente

            ruta.append(0)
            r += "0"
            print(r)
            rutas[camion] = ruta
            
print("\n==== Arreglo de rutas ====")
print(rutas)


==== Ruta camión 1 ====
0 -> 2 -> 11 -> 5 -> 15 -> 0
==== Ruta camión 2 ====
0 -> 17 -> 10 -> 6 -> 8 -> 0
==== Ruta camión 3 ====
0 -> 7 -> 16 -> 13 -> 20 -> 12 -> 0
==== Ruta camión 4 ====
0 -> 3 -> 18 -> 14 -> 0
==== Ruta camión 5 ====
0 -> 4 -> 1 -> 19 -> 9 -> 0

==== Arreglo de rutas ====
{1: [0, 2, 11, 5, 15, 0], 2: [0, 17, 10, 6, 8, 0], 3: [0, 7, 16, 13, 20, 12, 0], 4: [0, 3, 18, 14, 0], 5: [0, 4, 1, 19, 9, 0]}


In [220]:
fig = go.Figure()

# RUTAS
for id_camion, ruta in rutas.items():
    coords_x = [COORDENADAS[nodo]["x"] for nodo in ruta]
    coords_y = [COORDENADAS[nodo]["y"] for nodo in ruta]
    capacidad_ruta = sum(map(lambda x: DEMANDAS[x] if x != 0 else 0, ruta))
    distancia_ruta = sum([DISTANCIAS[(ruta[i], ruta[i + 1])] for i in range(len(ruta) - 1)])

    fig.add_trace(
        go.Scatter(
            x=coords_x,
            y=coords_y,
            mode="lines",
            line=dict(color=px.colors.qualitative.G10[id_camion % len(px.colors.qualitative.G10)], width=2, dash="solid"),
            name=f"Ruta C{id_camion} - CAP: {capacidad_ruta} D: {distancia_ruta:.2f}",
        )
    )

    # FLECHAS
    for i in range(len(ruta) - 1):
        mid_x = (coords_x[i] + coords_x[i + 1]) / 2
        mid_y = (coords_y[i] + coords_y[i + 1]) / 2
        fig.add_annotation(
            x=mid_x, y=mid_y,
            ax=mid_x - (coords_x[i + 1] - coords_x[i]) * 0.1, ay=mid_y - (coords_y[i + 1] - coords_y[i]) * 0.1,
            xref="x",
            yref="y",
            axref="x",
            ayref="y",
            showarrow=True,
            arrowhead=1,
            arrowsize=2,
            arrowwidth=1,
            arrowcolor=px.colors.qualitative.G10[id_camion % len(px.colors.qualitative.G10)],
        )

# DEPÓSITO
fig.add_trace(
    go.Scatter(
        x=[COORDENADAS[0]["x"]],
        y=[COORDENADAS[0]["y"]],
        mode="markers",
        name="Depósito",
        marker=dict(color="red", size=15),
    )
)

# CLIENTES
fig.add_trace(
    go.Scatter(
        x=[coord["x"] for coord in list(COORDENADAS.values())[1:]],
        y=[coord["y"] for coord in list(COORDENADAS.values())[1:]],
        mode="markers",
        name="Clientes",
        marker=dict(color="black", size=10),
    )
)

for cliente in CLIENTES:
    coord_cliente_x = COORDENADAS[cliente]["x"]
    coord_cliente_y = COORDENADAS[cliente]["y"]
    fig.add_annotation(x=coord_cliente_x - 8, y=coord_cliente_y + 8, text=f"<b>C{cliente}: </b> {DEMANDAS[cliente]}", showarrow=False)


fig.update_layout(
    title=f"RECORRIDO TOTAL: {solver.ObjectiveValue():.2f}",
    xaxis_title="X",
    yaxis_title="Y",
    xaxis=dict(range=[-10, N * 10 + 10]),
    yaxis=dict(range=[-10, N * 10 + 10]),
    width=800,
    height=600,
    template="ggplot2",
)

fig.show()